In [11]:
import pandas as pd
import numpy as np
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

# Q1. Downloading the data

In [2]:
df = pd.read_parquet("../../01-intro/data/yellow_tripdata_2023-01.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
df_january_2023 = df[
    (df['tpep_pickup_datetime'].dt.month == 1) & 
    (df['tpep_pickup_datetime'].dt.year == 2021)
]
df_january_2023.shape

(0, 19)

# Q2. Computing duration

In [7]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

df_january = df[df['tpep_pickup_datetime'].dt.month == 1]

std_duration = df_january['duration'].std()
print(std_duration)

42.585641764259044


# Q3. Dropping outliers

In [8]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

total_records = len(df)

df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

filtered_records = len(df_filtered)

fraction_remaining = filtered_records / total_records
print(fraction_remaining)

0.9812202822125979


# Q4. One-hot encoding

In [9]:
df_filtered.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [10]:
df_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

dv = DictVectorizer()

X = dv.fit_transform(df_dicts)

n_features = X.shape[1]

print(f"Dimensionality of the feature matrix: {n_features}")

Dimensionality of the feature matrix: 515


# Q5. Training a model

In [12]:
# Target variable
y_train = df_filtered['duration'].values

# Train linear regression model
model = LinearRegression()
model.fit(X, y_train)

# Predict on training set
y_pred = model.predict(X)

# Compute RMSE
rmse = mean_squared_error(y_train, y_pred, squared=False)

print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 7.65


/home/mlopszoomcamp/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Q6. Evaluating the model

In [14]:
df_val = pd.read_parquet('../../01-intro/data/yellow_tripdata_2023-02.parquet')  # or however you load it
df_val['tpep_pickup_datetime']  = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']) \
                      .dt.total_seconds() / 60
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

dicts_val = df_val[['PULocationID','DOLocationID']].astype(str) \
              .to_dict(orient='records')
X_val = dv.transform(dicts_val)

y_val    = df_val['duration'].values
y_pred_v = model.predict(X_val)

rmse_val = mean_squared_error(y_val, y_pred_v, squared=False)
print(f"Validation RMSE (Feb 2023): {rmse_val:.2f} minutes")

Validation RMSE (Feb 2023): 7.81 minutes


/home/mlopszoomcamp/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
